# Cadence Effects

This notebook simulates normal Type Ia Supernova (SN Ia) light-curves using realistic cadences and atmospheric variabilities expected from LSST.


In [ ]:
import sys
sys.path.insert(0, '../')

import sncosmo
from astropy.io import fits
from astropy.table import Table
from matplotlib import pyplot as plt
from pwv_kpno.package_settings import ConfigBuilder, settings
from pwv_kpno import pwv_atm

from sn_analysis import filters
from sn_analysis import plasticc
from sn_analysis import plotting

filters.register_lsst_filters(force=True)


In [ ]:
plt.rcParams['figure.dpi'] = 100


## Atmospheric Variability

To create a physically reasonable representation of the atmospheric variability at LSST, we use PWV measurements taken at the nearby Cerro Telolo International Observatory (CTIO).


In [ ]:
ctio_config = ConfigBuilder(
    site_name='cerro_tololo',
    primary_rec='CTIO',
    sup_rec=[]
)

ctio_config.save_to_ecsv('./cerro_tololo.ecsv', overwrite=True)
settings.import_site_config('./cerro_tololo.ecsv', overwrite=True)
settings.set_site('cerro_tololo')


In [ ]:
pwv_atm.update_models(range(2012, 2019))


In [ ]:
ctio_pwv = pwv_atm.measured_pwv().to_pandas('date')

ctio_pwv.reset_index().plot.scatter('date', 'CTIO', s=1, figsize=(10, 4), alpha=.2)
plt.ylabel('CTIO PWV (mm)')
plt.xlabel('Date')
plt.title('All available PWV measurements for CTIO')
plt.ylim(0, 20)


We don't have enough data to fully represent a 10 year long survey. Fortunately we are mostly interested in timescales of seasonal variability and shorter so we can consider data from a single year with good measurement coverage. 

In [ ]:
plotting.plot_year_pwv_vs_time(ctio_pwv[ctio_pwv.index.year == 2016].CTIO)
plt.title('CTIO PWV over 2016');


Out of simple curiosity, we also visualize the median PWV across all available years.


In [ ]:
good_pwv_data = ctio_pwv[ctio_pwv.CTIO <= 25]
good_pwv_data.index = good_pwv_data.index.map(lambda t: t.replace(year=2000))  # Use a dummy year
folded_pwv = good_pwv_data.groupby(good_pwv_data.index).CTIO.median()


In [ ]:
plotting.plot_year_pwv_vs_time(folded_pwv)
plt.title('Phase folded PWV');


## The PLaSTICC Data

Instead of evaluating different cadences from scratch, we use light-curves from the PLaSTICC simulations. First we check what cadence simulations are available on the notebook's host server.


In [ ]:
plasticc.get_available_cadences()


Simulated light-curves are written in the SNANA file format and are distributed across multiple files. We load a light-curve from one of these files and demosntrate the data model below. Each cadence includes simulations run with multiple supernova models. In this notebook we only need simulations for normal SNe (Model 11). 


In [ ]:
demo_cadence = 'alt_sched_rolling'
demo_cadence_header_files = plasticc.get_model_headers('alt_sched_rolling', 11)

demo_header_path = demo_cadence_header_files[0]
with fits.open(demo_header_path) as _temp:
    light_curves_per_file = len(_temp[1].data) 

print('Files per cadence:', len(demo_cadence_header_files))
print('Max light-curves per file:', light_curves_per_file)
    

In [ ]:
plasticc_lc = next(plasticc.iter_lc_for_header(demo_header_path, verbose=False))


In [ ]:
plasticc_lc.meta


In [ ]:
plasticc_lc


Here we reformat the data to be compatible with `sncosmo` so we can easily visualize the light-curve.


In [ ]:
formatted_lc = plasticc.format_plasticc_sncosmo(plasticc_lc)


In [ ]:
sncosmo.plot_lc(formatted_lc);


## Simulating Light-Curves

Since we need to add in our own atmospheric variability, the pre-tabulated flux values above are of limited use. Instead, we use the PLaSTICC light-curves to establish the cadence and model parameters for each simulated SN. This information is then used to simulate our own light-curves with `sncosmo`.


In [ ]:
duplicated_lc = plasticc.duplicate_plasticc_sncosmo(plasticc_lc)


In [ ]:
duplicated_lc.meta


In [ ]:
sncosmo.plot_lc(duplicated_lc);


In [ ]:
duplicated_lc


## Fitting Light-Curves

In [ ]:
def iter_custom_lcs(cadence, scatter=True, quality_callback=None, drop_nondetection=False, verbose=True):
    """Simulate light-curves for a given cadence
    
    Args:
        cadence               (str): Cadence to use when simulating light-curves
        scatter              (bool): Add random noise to the flux values
        quality_callback (callable): Skip light-curves if this function returns False
        drop_nondetection    (bool): Drop data with PHOTFLAG == 0
        verbose              (bool): Display a progress bar
    """
    
    cadence_header = plasticc.get_model_headers(cadence, model=11)[0]
    for light_curve in plasticc.iter_lc_for_header(cadence_header, verbose=verbose):
        
        lc = plasticc.duplicate_plasticc_sncosmo(light_curve, scatter=scatter)
        if quality_callback and not quality_callback(lc):
            continue
            
        yield lc 
        

In [ ]:
def passes_quality_cuts(light_curve):
    """Return whether light-curve has 2+ two bands each with 1+ data point with SNR > 5
    
    Args:
        light_curve (Table): Astropy table with sncosmo formatted light-curve data
        
    Returns:
        A boolean
    """
    
    light_curve = light_curve.group_by('band')
    
    passed_cuts = []
    for band_lc in light_curve.groups:
        passed_cuts.append((band_lc['flux'] /  band_lc['fluxerr'] > 5).any())
        
    return sum(passed_cuts) >= 2
        

In [ ]:
break  # Stop below cells from running automaticaly when running all cells

In [ ]:
bands = ['lsst_hardware_' + b for b in 'ugrizy']

# Iterator over simulated light-curves
light_curves = iter_custom_lcs('alt_sched_rolling', quality_callback=passes_quality_cuts)

# Fit light curves
vparams = ['x0', 'x1', 'c']
fitted_mag, fitted_params = sn_magnitudes.fit_mag(
        'salt2', light_curves, vparams, bands=bands)
